In [1]:
"""
IMPORTS
"""
import pandas as pd
import xlrd
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

In [6]:
df_2020_to_resample=pd.read_csv("E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Incomes\servicios_sociales_urgencias_2020.csv",sep=';',encoding='utf-8',index_col='Secuencia')


In [5]:
df_2020_to_resample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24351 entries, 1 to 24351
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Código Centro                 24351 non-null  int64  
 1   Centro                        24351 non-null  object 
 2   Código Distrito Centro        24351 non-null  int64  
 3   Distrito Centro               24351 non-null  object 
 4   Código Distrito               24351 non-null  int64  
 5   Distrito                      24351 non-null  object 
 6   Código Barrio                 24351 non-null  int64  
 7   Barrio                        24351 non-null  object 
 8   Sección Censal                24351 non-null  int64  
 9   Tramo Edad                    24351 non-null  object 
 10  Nacionalidad                  24351 non-null  object 
 11  Sexo                          24351 non-null  object 
 12  Código Tipo Atención          24351 non-null  int64  
 13  T

Vamos a eliminar todas las columnas que no sean validas.

In [7]:
df_2020_to_resample.drop(['Centro','Distrito Centro','Código Distrito','Distrito','Código Barrio','Barrio','Sección Censal'],axis=1,inplace=True) 

In [8]:
df_2020_to_resample

,Código Centro,Código Distrito Centro,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita
Secuencia,,,,,,,,,,
1,69,2,50 - 54,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,02/01/2020
2,69,2,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,02/01/2020
3,69,2,60 - 64,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,02/01/2020
4,69,2,75 - 79,Española,H,1,Primera Atención,40.0,Desproteccion social problema salud,02/01/2020
5,99,21,40 - 44,Española,M,2,Zona / Historia Social,50.0,Desprotección socio-familiar,02/01/2020
...,...,...,...,...,...,...,...,...,...,...
24347,130,16,>= 85,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,30/12/2020
24348,86,13,25 - 29,Extranjera,M,1,Primera Atención,30.0,Alojamiento/Vivienda,30/12/2020
24349,86,13,>= 85,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,30/12/2020


In [9]:
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace(' - ','-')
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace('- ','-')
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace('= ','=')
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A4-11", "<=18")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A12-17", "<=18")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A0-3", "<=18") 

df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A18-24", "19-29")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A25-29", "19-29")

df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A30-34", "30-39")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A35-39", "30-39")

df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A40-44", "40-49")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A44-49", "40-49")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A45-49", "40-49")

df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A50-54", "50-59")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A55-59", "50-59")

df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A60-64", "60-64")

df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A65-69", ">=65")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A75-79", ">=65")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A80-84", ">=65")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A70-74", ">=65")
df_2020_to_resample['Tramo Edad']=df_2020_to_resample['Tramo Edad'].str.replace("\A>=85", ">=65")

c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
c:\Users\jtr_7\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future ve

In [10]:
lista_rango_edad=df_2020_to_resample['Tramo Edad'].unique()                  # Hay que revisar esto, porque no está ordenado y sería lo ideal
lista_rango_edad.sort()
mivalor = [ x for x in range(len(lista_rango_edad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
mivalor
lista_rango_edad=list(lista_rango_edad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 

Age_range_code = {k: v for k, v in zip(lista_rango_edad, mivalor)}   ##  sustituirlo por cero
Age_range_code
df_2020_to_resample['Age_range_code']= df_2020_to_resample['Tramo Edad'].map(Age_range_code)

In [11]:
lista_nacionalidad=df_2020_to_resample['Nacionalidad'].unique()
lista_nacionalidad.sort()
lista_nacionalidad
mivalor = [ x for x in range(len(lista_nacionalidad))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_nacionalidad=list(lista_nacionalidad)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 
Nationality_code = {k: v for k, v in zip(lista_nacionalidad, mivalor)}   ##  sustituirlo por cero
df_2020_to_resample['Nationality_code']= df_2020_to_resample['Nacionalidad'].map(Nationality_code)

In [12]:
df_2020_to_resample['Sexo']=df_2020_to_resample['Sexo'].mask(df_2020_to_resample['Sexo'] == ' ').fillna('M')

In [13]:
lista_sexo=df_2020_to_resample['Sexo'].unique()
lista_sexo.sort()
lista_sexo
mivalor = [ x for x in range(len(lista_sexo))]             ##  agregaremos una variable numérica para sustituir a los nombres de las cias
#mivalor    
lista_sexo=list(lista_sexo)                               ##  dejamos el valor cero por si en los datos a testear  hay algún NaN y 

Sex_code = {k: v for k, v in zip(lista_sexo, mivalor)}   ##  sustituirlo por cero

df_2020_to_resample['Sex_code']= df_2020_to_resample['Sexo'].map(Sex_code)

In [15]:
df_2020_to_resample['Fecha Cita'] = pd.to_datetime(df_2020_to_resample['Fecha Cita'], dayfirst=True)

df_2020_to_resample['year'] = df_2020_to_resample['Fecha Cita'].dt.year
#df_demandas['month'] = df_demandas['Timestamp'].dt.month
df_2020_to_resample['Quarteryear'] = df_2020_to_resample['Fecha Cita'].dt.quarter
df_2020_to_resample

,Código Centro,Código Distrito Centro,Tramo Edad,Nacionalidad,Sexo,Código Tipo Atención,Tipo Atención,Código Tipo Supuesto Urgente,Tipo Supuesto Urgente,Fecha Cita,Age_range_code,Nationality_code,Sex_code,year,Quarteryear
Secuencia,,,,,,,,,,,,,,,
1,69,2,50-59,Española,H,1,Primera Atención,50.0,Desprotección socio-familiar,2020-01-02,4,0,0,2020,1
2,69,2,>=65,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,2020-01-02,7,0,1,2020,1
3,69,2,60-64,Extranjera,H,2,Zona / Historia Social,30.0,Alojamiento/Vivienda,2020-01-02,5,1,0,2020,1
4,69,2,>=65,Española,H,1,Primera Atención,40.0,Desproteccion social problema salud,2020-01-02,7,0,0,2020,1
5,99,21,40-49,Española,M,2,Zona / Historia Social,50.0,Desprotección socio-familiar,2020-01-02,3,0,1,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24347,130,16,>=65,Española,M,2,Zona / Historia Social,40.0,Desproteccion social problema salud,2020-12-30,7,0,1,2020,4
24348,86,13,19-29,Extranjera,M,1,Primera Atención,30.0,Alojamiento/Vivienda,2020-12-30,1,1,1,2020,4
24349,86,13,>=65,Española,M,1,Primera Atención,40.0,Desproteccion social problema salud,2020-12-30,7,0,1,2020,4


In [16]:
df_2020_to_resample['Tipo Supuesto Urgente'].isnull().sum()

1402

In [17]:
df_2020_to_resample['Código Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
df_2020_to_resample['Tipo Supuesto Urgente'].fillna(method="ffill",inplace=True)
'''Verificamos'''
print("Los valores NaN que hay en este Dataset son: ",df_2020_to_resample['Código Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2020_to_resample))
print("Los valores NaN que hay en este Dataset son: ",df_2020_to_resample['Tipo Supuesto Urgente'].isnull().sum())
print("Los registros que hay en este DataSet son: ",len(df_2020_to_resample))
df_2020_to_resample.groupby('Código Tipo Supuesto Urgente')['Tipo Supuesto Urgente'].value_counts()

Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  24351
Los valores NaN que hay en este Dataset son:  0
Los registros que hay en este DataSet son:  24351


Código Tipo Supuesto Urgente  Tipo Supuesto Urgente              
10.0                          Malos Tratos                             290
20.0                          Abandono                                  23
30.0                          Alojamiento/Vivienda                    2016
40.0                          Desproteccion social problema salud     4539
50.0                          Desprotección socio-familiar           17483
Name: Tipo Supuesto Urgente, dtype: int64

Vamos a ver como son los valores de la variable target para ver como está de desbalanceado y aplicar las correcciones oportunas.

In [18]:
df_2020_to_resample['Código Tipo Supuesto Urgente'].value_counts()

50.0    17483
40.0     4539
30.0     2016
10.0      290
20.0       23
Name: Código Tipo Supuesto Urgente, dtype: int64

In [19]:
X=df_2020_to_resample[['Código Distrito Centro','Age_range_code','Nationality_code','Sex_code','year']]
y=df_2020_to_resample['Código Tipo Supuesto Urgente']

In [20]:
strategy_u = {50.0: 743, 40.0:2236, 30.0:2016, 20.0:23, 10.0:290}
#strategy_o = {50.0: 4000, 40.0:3000, 30.0:2500, 20.0:2344, 10.0:3000}

In [21]:
under=RandomUnderSampler(sampling_strategy=strategy_u)
#over=SMOTE(sampling_strategy=strategy_o)
steps=[('u',under)]#,('o',over)]
pipeline= Pipeline(steps=steps)
X,y=pipeline.fit_resample(X,y)

In [22]:
df_2020=pd.DataFrame()
df_2020['Código Distrito Centro']=X['Código Distrito Centro']
df_2020['Age_range_code']=X['Age_range_code']
df_2020['Nationality_code']=X['Nationality_code']
df_2020['Sex_code']=X['Sex_code']
df_2020['year']=X['year']
df_2020['Código Tipo Supuesto Urgente']=y
df_2020



,Código Distrito Centro,Age_range_code,Nationality_code,Sex_code,year,Código Tipo Supuesto Urgente
0,10,2,1,1,2020,10.0
1,6,3,1,0,2020,10.0
2,6,2,1,1,2020,10.0
3,13,3,0,1,2020,10.0
4,6,3,0,1,2020,10.0
...,...,...,...,...,...,...
5303,8,2,1,1,2020,50.0
5304,6,4,1,0,2020,50.0
5305,8,7,0,0,2020,50.0
5306,6,1,1,1,2020,50.0


In [23]:
df_2020['Código Tipo Supuesto Urgente'].value_counts()

40.0    2236
30.0    2016
50.0     743
10.0     290
20.0      23
Name: Código Tipo Supuesto Urgente, dtype: int64

In [24]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5308 entries, 0 to 5307
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Código Distrito Centro        5308 non-null   int64  
 1   Age_range_code                5308 non-null   int64  
 2   Nationality_code              5308 non-null   int64  
 3   Sex_code                      5308 non-null   int64  
 4   year                          5308 non-null   int64  
 5   Código Tipo Supuesto Urgente  5308 non-null   float64
dtypes: float64(1), int64(5)
memory usage: 248.9 KB


In [31]:
df_2020.to_csv('E:\Bootcamp_22\Javier\Repositorios\Machine_Learning\data\Outcomes\df_2020_balanced.csv',header=True,index=False)